In [1]:
import cv2 as cv
import matplotlib.pyplot as plt
import numpy as np
from ipywidgets import interact

In [2]:
img = cv.imread('imgs/tree.png')
img = cv.cvtColor(img, cv.COLOR_BGR2RGB)
img_gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)

## Binarization and Segmentation

We can isolate brighter and darker colour by applying a threshold to the grayscal image, and using the result as a mask on the original image

In [4]:
# 138 obscures the tree
# 175 isolates the flower
@interact(thr=(0,255), invert=[True,False])
def images_thr(thr, invert):
    # img_thr = img_gray.copy()
    bin_filter = img_gray > thr
    if invert:
        bin_filter = np.logical_not(bin_filter)
    
    # img_thr[bin_filter] = 255
    # img_thr[np.logical_not(bin_filter)] = 0
    img_thr = np.logical_and(img_gray, bin_filter) *255

    img_segmented = img.copy()
    img_segmented[np.logical_not(bin_filter)] = 0

    fig, (ax1,ax2,ax3,ax4) = plt.subplots(1,4, figsize=(10,10))
    ax1.imshow(img, vmin=0, vmax=255)
    ax1.set_axis_off()
    ax1.set_title('Original')
    ax2.imshow(img_gray, cmap='gray',vmin=0, vmax=255)
    ax2.set_axis_off()
    ax2.set_title('Gray')
    ax3.imshow(img_thr, cmap='gray',vmin=0, vmax=255)
    ax3.set_axis_off()
    ax3.set_title('Threshold Mask')
    ax4.imshow(img_segmented, cmap='gray',vmin=0, vmax=255)
    ax4.set_axis_off()
    ax4.set_title('Segmented Image')

interactive(children=(IntSlider(value=127, description='thr', max=255), Dropdown(description='invert', options…

## Convolutions

In [8]:
n_pixels = len(img[:,:,0].ravel())
@interact(progress=(0,n_pixels), pad_mode = ['constant', 'edge', 'reflect'])
def convolution(progress, pad_mode):
    kernel = np.array([
        [1/9, 1/9, 1/9], 
        [1/9, 1/9, 1/9],
        [1/9, 1/9, 1/9]
    ])
    # apply the convolution
    img_transformed = img.copy()
    img_pad = np.zeros_like(img, shape=(img.shape[0]+2, img.shape[1]+2,3))
    for i in range(3):
        img_pad[:,:,i] = np.pad(img[:,:,i], 1, mode=pad_mode)
    
    for i in range(img.shape[0]):
        j=0
        if i*img.shape[1] + j >= progress:
                break
        for j in range(img.shape[1]):
            if i*img.shape[1] + j >= progress:
                break
            for s in range(3):
                img_transformed[i,j, s] = (img_pad[i:i+3, j:j+3, s] * kernel).sum()
    fig, (ax1,ax2,ax3) = plt.subplots(1,3, figsize=(10,10))
    ax1.imshow(img, vmin=0, vmax=255)
    ax1.set_axis_off()
    ax1.set_title('Original')
    ax2.imshow(img_pad, cmap='gray',vmin=0, vmax=255)
    ax2.set_axis_off()
    ax2.set_title('Padded')
    ax3.imshow(img_transformed, cmap='gray',vmin=0, vmax=255)
    ax3.set_axis_off()
    ax3.set_title('Transformed')



interactive(children=(IntSlider(value=50, description='progress'), Dropdown(description='pad_mode', options=('…